In [3]:
import numpy as np
import pandas as pd
from tqdm import tqdm
import pickle
import cv2

In [4]:
resnet50_match_distance = pd.read_csv('resnet50_matches_distance.csv')
resnet50_match_filename = pd.read_csv('resnet50_matches_filename.csv')

In [5]:
train_i = []
test_i = []

train_xy = pd.read_csv('train.csv')
train_path = train_xy['id'].values
test_path = pd.read_csv('imagenames1.csv')['id'].values

for f in train_path:
    i = cv2.imread('./train/' + f + '.jpg')
    train_i.append(i)
for f in test_path:
    i = cv2.imread('./test/' + f + '.jpg')
    test_i.append(i)

1200

In [9]:
sift = cv2.SIFT_create()


for i,img in tqdm(enumerate(test_i)):
    _, des = sift.detectAndCompute(img,None)
    f = open(f'./test_kp1/test_kp{test_path[i]}.pckl','wb')
    pickle.dump(des,f)
    f.close()


1200it [02:21,  8.51it/s]


In [23]:
FLANN_INDEX_KDTREE = 1
index_params = dict(algorithm = FLANN_INDEX_KDTREE, trees = 5)
search_params = dict(checks=50)
flann = cv2.FlannBasedMatcher(index_params,search_params)

i = 0
match=[]
no_match =[]
for test in tqdm(test_path):

    f = open(f'./test_kp1/test_kp{test}.pckl', 'rb')
    des_test = pickle.load(f)
    f.close()

    goods = []
    current_good = 0

    if des_test is None or len(des_test) < 2:
        no_match.append(test)
        continue

    for train in resnet50_match_filename.loc[i][:20]:
        f = open(f'./train_kp/train_kp{train}.pckl', 'rb')
        des_train = pickle.load(f)
        f.close()

        # Matching descriptor using KNN algorithm
        if des_train is None or len(des_train) < 2:
            goods.append(-1)
            continue
        matches = flann.knnMatch(des_test,des_train,k=2)

        # Store all good matches as per Lowe's Ratio test.
        good = len([m for m,n in matches if m.distance < 0.7*n.distance])
        goods.append(good)
    
    sorted_idx = np.argsort(goods)
    sorted_path = [resnet50_match_filename.loc[i][idx] for idx in sorted_idx]
    sorted_goods = [goods[idx] for idx in sorted_idx]
    i +=1
#     print(sorted_goods)
#     print(sorted_path)
    match.append((test,sorted_path[-1]))
    

100%|██████████████████████████████████████████████████████████████████████████████| 1200/1200 [16:50<00:00,  1.19it/s]


In [41]:
match[0][0]
np.argwhere(test_path==no_match[0])[0][0]

81

In [54]:
out = []
for img in no_match:
    index = np.argwhere(test_path==img)[0][0]
    resnet50_match_filename.loc[index][:3]
    coor = [train_xy[train_xy['id'] == filename] for filename in resnet50_match_filename.loc[index][:3]]
    result = pd.concat(coor)
    out.append([img, np.mean(result['x']), np.mean(result['y'])])

In [55]:
for i in range(len(match)):
    coor = train_xy[train_xy['id']== match[i][1]]
    out.append([match[i][0],coor.iloc[0]['x'],coor.iloc[0]['y']])
out_csv = pd.DataFrame(out, columns=['id','x','y'])
out_csv.to_csv('resnet50_sift_top1_20_out.csv',index=False)